In [1]:
import pandas as pd
import numpy as np
import requests
import os
import json
from pprint import pprint

In [96]:
# Importing the df from city_bikes.ipynb
file_path = r"C:\Users\akash\Desktop\BootCamp\Project_2\Statistical-Modelling-with-Python-main\data\bike_station_info.csv"

# Read the CSV file into a DataFrame
city_bike_df = pd.read_csv(file_path)

In [97]:
city_bike_df

,city,station_id,station,latitude,longitude,slots,empty_slots,free_bikes,station_near_from
0,Saguenay,b6903d788830595e054121e462598381,Place du Citoyen,48.428023,-71.062506,7,4,2,"48.428023, -71.062506"
1,Saguenay,3aa299f4d3048ef94737bbf5fbc03825,Place du Royaume,48.404457,-71.058208,7,1,6,"48.404457, -71.058208"
2,Saguenay,0b1c0c48e94fae24b1a38f4559a5985a,UQAC,48.420480,-71.053380,7,5,2,"48.420480, -71.053380"
3,Saguenay,f450af93c8d0f5ae5de8ea7cb1aafe70,Terminus La Baie,48.337230,-70.879564,7,5,2,"48.337230, -70.879564"
4,Saguenay,7ada5d893c40c9a46a54f771a392db6f,Cégep de Chicoutimi,48.425119,-71.051336,7,5,2,"48.425119, -71.051336"
5,Saguenay,989fc50e15bad6b1f40d26f986d50ccc,Station Rene-Levesque,48.404255,-71.216372,7,7,0,"48.404255, -71.216372"
6,Saguenay,e5b2a5bf7b36bcd5d52ca0d63a56ae06,Terminus Jonquiere,48.411807,-71.254890,7,3,4,"48.411807, -71.254890"


# Foursquare

### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [98]:
# Getting information of all places within Saguenay, CA

api_key = os.environ["FOURSQUARE_API_KEY"]
location = "Saguenay"
radius = 1000
url = "https://api.foursquare.com/v3/places/search?near=" + location

# Create dictionary for headers
headers = {"Accept": "application/json"}
# Add key with our API KEY
headers['Authorization'] = api_key

result = requests.get(url, headers=headers)

# Checking if the request was successful
if result.status_code == 200:
    # Filter the results based on the maximum radius
    data = result.json()
    filtered_results = []
    for result in data["results"]:
        if "distance" in result and result["distance"] <= radius:
            filtered_results.append(result)
    pprint(filtered_results)
    
else:
    print(f"Error occurred: {response.status_code} - {response.json()['error']['message']}")

[{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
                           'suffix': '.png'},
                  'id': 16032,
                  'name': 'Park'}],
  'chains': [],
  'distance': 283,
  'fsq_id': '4f2d66cf0039af30f2ac2c1d',
  'geocodes': {'drop_off': {'latitude': 48.41811, 'longitude': -71.062655},
               'main': {'latitude': 48.418308, 'longitude': -71.062855}},
  'link': '/v3/places/4f2d66cf0039af30f2ac2c1d',
  'location': {'address': '700 Begin Rue',
               'country': 'CA',
               'formatted_address': '700 Begin Rue, Chicoutimi QC G7H 7Y6',
               'locality': 'Chicoutimi',
               'postcode': 'G7H 7Y6',
               'region': 'QC'},
  'name': 'Parc Rosaire-Gauthier',
  'related_places': {},
  'timezone': 'America/Toronto'},
 {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
                           'suffix': '.png'},
     

### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc) and Put your parsed results into a DataFrame

In [99]:
# creating a data frame with the values of a JSON file.

unique_places = []
pois = data.get("results", [])
for poi in pois:
    categories = poi.get("categories", [])
    geocodes = poi.get("geocodes", {})
    main_geocode = geocodes.get("main", {})
    for category in categories:
        category_id = category.get("id")
        place_name = poi.get("name")
        category_name = category.get("name")
        place_address = poi.get("location", {}).get("formatted_address")
        rating = poi.get("rating")
        distance = poi.get("distance")
        latitude = main_geocode.get("latitude")
        longitude = main_geocode.get("longitude")
        unique_places.append({"category_id": category_id, 
                              "place_name": place_name, 
                              "category_name": category_name, 
                              "address": place_address, 
                              "rating": rating,
                              "latitude": latitude,
                              "longitude": longitude
                             })

        
# Create DataFrame
df_unique = pd.DataFrame(unique_places)

# Print the DataFrame
df_unique

,category_id,place_name,category_name,address,rating,latitude,longitude
0,16032,Parc Rosaire-Gauthier,Park,"700 Begin Rue, Chicoutimi QC G7H 7Y6",None,48.418308,-71.062855
1,10000,Club de Curling de Chicoutimi,Arts and Entertainment,"760 Jolliet Rue, Chicoutimi QC G7J 2P8",None,48.418273,-71.069723
2,12051,École de conduite Tecnic Chicoutimi,Driving School,"87 des Oblats Rue E, Chicoutimi QC G7H 2X9",None,48.414023,-71.067610
3,15026,Lambert Sport Physio,Physical Therapy Clinic,"990B Begin Rue (Rue Morin), Chicoutimi QC G7H 5X2",None,48.412863,-71.068259
4,10051,Georges-Vézina Ctr,Stadium,"643 Begin Rue (Blvd Université), Chicoutimi QC...",None,48.419693,-71.059928
5,13276,Aki Sushi,Sushi Restaurant,"993 Talbot Blvd, Chicoutimi QC G7H 4B5",None,48.412394,-71.049634
6,10030,La Pulperie de Chicoutimi,History Museum,"300 Dubuc Rue, Saguenay QC G7J 4M1",None,48.422030,-71.081402
7,13016,La Voie Maltée,Lounge,"777 Talbot Blvd, Chicoutimi QC G7H 4B3",None,48.416622,-71.045653
8,13029,La Voie Maltée,Brewery,"777 Talbot Blvd, Chicoutimi QC G7H 4B3",None,48.416622,-71.045653
9,13383,La Voie Maltée,Steakhouse,"777 Talbot Blvd, Chicoutimi QC G7H 4B3",None,48.416622,-71.045653


df_unique shows all the places in the city Saguenay, CA. Includes all the POIs.

### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [100]:
# Station_near_from column retrives from city_bike_df, the reason for this is to get infomation of which POI is near which bike station.

df_unique['station_near_from'] = city_bike_df.apply(lambda row: f"{row['latitude']:.6f}, {row['longitude']:.6f}", axis=1)
df_unique

,category_id,place_name,category_name,address,rating,latitude,longitude,station_near_from
0,16032,Parc Rosaire-Gauthier,Park,"700 Begin Rue, Chicoutimi QC G7H 7Y6",None,48.418308,-71.062855,"48.428023, -71.062506"
1,10000,Club de Curling de Chicoutimi,Arts and Entertainment,"760 Jolliet Rue, Chicoutimi QC G7J 2P8",None,48.418273,-71.069723,"48.404457, -71.058208"
2,12051,École de conduite Tecnic Chicoutimi,Driving School,"87 des Oblats Rue E, Chicoutimi QC G7H 2X9",None,48.414023,-71.067610,"48.420480, -71.053380"
3,15026,Lambert Sport Physio,Physical Therapy Clinic,"990B Begin Rue (Rue Morin), Chicoutimi QC G7H 5X2",None,48.412863,-71.068259,"48.337230, -70.879564"
4,10051,Georges-Vézina Ctr,Stadium,"643 Begin Rue (Blvd Université), Chicoutimi QC...",None,48.419693,-71.059928,"48.425119, -71.051336"
5,13276,Aki Sushi,Sushi Restaurant,"993 Talbot Blvd, Chicoutimi QC G7H 4B5",None,48.412394,-71.049634,"48.404255, -71.216372"
6,10030,La Pulperie de Chicoutimi,History Museum,"300 Dubuc Rue, Saguenay QC G7J 4M1",None,48.422030,-71.081402,"48.411807, -71.254890"
7,13016,La Voie Maltée,Lounge,"777 Talbot Blvd, Chicoutimi QC G7H 4B3",None,48.416622,-71.045653,NaN
8,13029,La Voie Maltée,Brewery,"777 Talbot Blvd, Chicoutimi QC G7H 4B3",None,48.416622,-71.045653,NaN
9,13383,La Voie Maltée,Steakhouse,"777 Talbot Blvd, Chicoutimi QC G7H 4B3",None,48.416622,-71.045653,NaN


In [101]:
# Creating clean df for visual purpose
foursquare_api = df_unique[['category_id', 'place_name', 'category_name', 'station_near_from']]

# Print the new DataFrame
foursquare_api

,category_id,place_name,category_name,station_near_from
0,16032,Parc Rosaire-Gauthier,Park,"48.428023, -71.062506"
1,10000,Club de Curling de Chicoutimi,Arts and Entertainment,"48.404457, -71.058208"
2,12051,École de conduite Tecnic Chicoutimi,Driving School,"48.420480, -71.053380"
3,15026,Lambert Sport Physio,Physical Therapy Clinic,"48.337230, -70.879564"
4,10051,Georges-Vézina Ctr,Stadium,"48.425119, -71.051336"
5,13276,Aki Sushi,Sushi Restaurant,"48.404255, -71.216372"
6,10030,La Pulperie de Chicoutimi,History Museum,"48.411807, -71.254890"
7,13016,La Voie Maltée,Lounge,NaN
8,13029,La Voie Maltée,Brewery,NaN
9,13383,La Voie Maltée,Steakhouse,NaN


In [102]:
foursquare_api_final = pd.merge(city_bike_df, foursquare_api, on=['station_near_from'], how='left') # join the foursquare_api_final df with city_bike df, using left join to get all the infomarion includes in city_bike
foursquare_api_final.dropna(axis=1, how='all', inplace=True) #dropping empty columns
foursquare_api_final

,city,station_id,station,latitude,longitude,slots,empty_slots,free_bikes,station_near_from,category_id,place_name,category_name
0,Saguenay,b6903d788830595e054121e462598381,Place du Citoyen,48.428023,-71.062506,7,4,2,"48.428023, -71.062506",16032,Parc Rosaire-Gauthier,Park
1,Saguenay,3aa299f4d3048ef94737bbf5fbc03825,Place du Royaume,48.404457,-71.058208,7,1,6,"48.404457, -71.058208",10000,Club de Curling de Chicoutimi,Arts and Entertainment
2,Saguenay,0b1c0c48e94fae24b1a38f4559a5985a,UQAC,48.420480,-71.053380,7,5,2,"48.420480, -71.053380",12051,École de conduite Tecnic Chicoutimi,Driving School
3,Saguenay,f450af93c8d0f5ae5de8ea7cb1aafe70,Terminus La Baie,48.337230,-70.879564,7,5,2,"48.337230, -70.879564",15026,Lambert Sport Physio,Physical Therapy Clinic
4,Saguenay,7ada5d893c40c9a46a54f771a392db6f,Cégep de Chicoutimi,48.425119,-71.051336,7,5,2,"48.425119, -71.051336",10051,Georges-Vézina Ctr,Stadium
5,Saguenay,989fc50e15bad6b1f40d26f986d50ccc,Station Rene-Levesque,48.404255,-71.216372,7,7,0,"48.404255, -71.216372",13276,Aki Sushi,Sushi Restaurant
6,Saguenay,e5b2a5bf7b36bcd5d52ca0d63a56ae06,Terminus Jonquiere,48.411807,-71.254890,7,3,4,"48.411807, -71.254890",10030,La Pulperie de Chicoutimi,History Museum


In [103]:
# Save the DataFrame as a CSV file in the data folder

folder_path = r"C:\Users\akash\Desktop\BootCamp\Project_2\Statistical-Modelling-with-Python-main\data"
file_name = "foursquare_api.csv"
file_path = os.path.join(folder_path, file_name)
foursquare_api.to_csv(file_path, index=False)

# Yelp

### Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [104]:
import requests
from pprint import pprint

yelp_api_key = os.environ["YELP_API_KEY"]

latitude = 48.412394
longitude = -71.059928

url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius=1000&categories=&sort_by=best_match&limit=5"

# Create dictionary for headers
headers = {
    "Authorization": f"Bearer {yelp_api_key}",
    "Accept": "application/json"
}

result = requests.get(url, headers=headers)
print(0)
pprint(result.json())
data = result.json() # storing JSON file to data

0
{'businesses': [{'alias': 'le-légendaire-restaurant-bar-chicoutimi',
                 'categories': [{'alias': 'french', 'title': 'French'}],
                 'coordinates': {'latitude': 48.410642952545814,
                                 'longitude': -71.05336150243492},
                 'display_phone': '+1 418-543-6120',
                 'distance': 522.3112533907296,
                 'id': 'hC4n_daXhCnoLeYofkjzaw',
                 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/pCgReAhLVWkn9ezxmXphbQ/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '1080 Boulevard Talbot',
                              'address2': '',
                              'address3': '',
                              'city': 'Chicoutimi',
                              'country': 'CA',
                              'display_address': ['1080 Boulevard Talbot',
                                                  'Chicoutimi, QC G7H 4B6',
                          

### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc) and Put your parsed results into a DataFrame

In [105]:
yelp_unique_places = []
businesses = data['businesses']

for business in businesses:
    name = business["name"]
    address = ", ".join(business["location"]["display_address"])
    latitude = business["coordinates"]["latitude"]
    longitude = business["coordinates"]["longitude"]
    country = business["location"]["country"]
    rating = business["rating"]
    review = business["review_count"]
    state = business["location"]["state"]

    yelp_unique_places.append({
        "name": name,
        "latitude": latitude,
        "longitude": longitude,
        "yelp_rating": rating,
        "yelp_review": review,
    })
        
# Create DataFrame
df_yelp_unique = pd.DataFrame(yelp_unique_places)

# Print the DataFrame
df_yelp_unique

,name,latitude,longitude,yelp_rating,yelp_review
0,Le Légendaire Restaurant Bar,48.410643,-71.053362,4.0,6
1,Boefish,48.405576,-71.057046,4.5,5
2,Passion Café,48.409290,-71.054947,4.0,5
3,Rôtisserie St-Hubert,48.413318,-71.049191,2.5,4
4,Ben & Florentine,48.403481,-71.054605,2.5,6


In [106]:
# Save the DataFrame as a CSV file in the data folder

folder_path = r"C:\Users\akash\Desktop\BootCamp\Project_2\Statistical-Modelling-with-Python-main\data"
file_name = "yelp_api.csv"
file_path = os.path.join(folder_path, file_name)
df_yelp_unique.to_csv(file_path, index=False)

# Comparing Results

### Which API provided you with more complete data? Provide an explanation. 

1) YELP_API pros:

- I thoroughly enjoy working with the Yelp API as it offers a user-friendly experience with clear documentation, intuitive integration, comprehensive business data and extensive user-generated reviews. 
- The platform's focus on local businesses provides valuable insights for location-specific information.
- Yelp's rich content, including photos and menus, enhances the user experience. 
- Additionally, the active user engagement, with features like posting reviews and ratings, contributes to a sense of community and credibility. 

Overall, the Yelp API offers a robust solution for accessing detailed business information and leveraging the power of user feedback.

2) FOURSQUARE_API pros:

- Foursquare API has distinct advantages over Yelp, including no rate limiting, a direct and structured data format, expanded search capabilities, and a developer-focused approach. 
- With higher rate limits and more flexible data access, Foursquare allows for faster and more extensive data retrieval, offering a wider range of information beyond just places.

In [107]:
df_unique.head(5)

,category_id,place_name,category_name,address,rating,latitude,longitude,station_near_from
0,16032,Parc Rosaire-Gauthier,Park,"700 Begin Rue, Chicoutimi QC G7H 7Y6",None,48.418308,-71.062855,"48.428023, -71.062506"
1,10000,Club de Curling de Chicoutimi,Arts and Entertainment,"760 Jolliet Rue, Chicoutimi QC G7J 2P8",None,48.418273,-71.069723,"48.404457, -71.058208"
2,12051,École de conduite Tecnic Chicoutimi,Driving School,"87 des Oblats Rue E, Chicoutimi QC G7H 2X9",None,48.414023,-71.067610,"48.420480, -71.053380"
3,15026,Lambert Sport Physio,Physical Therapy Clinic,"990B Begin Rue (Rue Morin), Chicoutimi QC G7H 5X2",None,48.412863,-71.068259,"48.337230, -70.879564"
4,10051,Georges-Vézina Ctr,Stadium,"643 Begin Rue (Blvd Université), Chicoutimi QC...",None,48.419693,-71.059928,"48.425119, -71.051336"


Getting the rating value and other in detail values of the POIs could not be possible with Foursquare, even using fsq_id.

In [108]:
df_yelp_unique.head(5)

,name,latitude,longitude,yelp_rating,yelp_review
0,Le Légendaire Restaurant Bar,48.410643,-71.053362,4.0,6
1,Boefish,48.405576,-71.057046,4.5,5
2,Passion Café,48.409290,-71.054947,4.0,5
3,Rôtisserie St-Hubert,48.413318,-71.049191,2.5,4
4,Ben & Florentine,48.403481,-71.054605,2.5,6


It seems the Yelp API has access to reviews and ratings, it is widely recognized as a leading platform for user-generated reviews and ratings, making it a valuable source of information for businesses.

### Get the top 10 restaurants according to their rating

In [109]:
# Easiest way to get this is using Yelp_api.
# Getting top 10 restaurents according to theier retings from Vancouver, CA.

yelp_api_key = os.environ["YELP_API_KEY"]

latitude = 48.419693
longitude = -71.059928

url = "https://api.yelp.com/v3/businesses/search?latitude=49.262487&longitude=-123.114397&radius=1000&categories=restaurants&sort_by=rating&limit=10"

# Create dictionary for headers
headers = {
    "Authorization": f"Bearer {yelp_api_key}",
    "Accept": "application/json"
}

result = requests.get(url, headers=headers)
top_10_data = result.json()

yelp_top_rest = []
businesses = top_10_data['businesses']

for business in businesses:
    name = business["name"]
    address = ", ".join(business["location"]["display_address"])
    latitude = business["coordinates"]["latitude"]
    longitude = business["coordinates"]["longitude"]
    country = business["location"]["country"]
    rating = business["rating"]
    review = business["review_count"]
    state = business["location"]["state"]

    yelp_top_rest.append({
        "name": name,
        "address": address,
        "latitude": latitude,
        "longitude": longitude,
        "country": country,
        "rating": rating,
        "review": review,
        "state": state
    })
        
# Create DataFrame
df_yelp_top_rest = pd.DataFrame(yelp_top_rest)

# Print the DataFrame
df_yelp_top_rest

,name,address,latitude,longitude,country,rating,review,state
0,The Golden Horn Turkish Bakery & Cafe,"2857 Cambie Street, Vancouver, BC V5Z 3Y8, Canada",49.259811,-123.115283,CA,5.0,9,BC
1,Jitlada,"118 W 6th Avenue, Vancouver, BC V5Y 1K6, Canada",49.265494,-123.107162,CA,5.0,4,BC
2,Convivial Cafe and Bakery,"680A Leg In Boot Square, Vancouver, BC V5Z 4B4...",49.267543,-123.119664,CA,4.5,11,BC
3,La Taqueria Food Truck,"Vancouver, BC, Canada",49.261226,-123.113927,CA,4.5,7,BC
4,Happy Wood BBQ House,"967 West Broadway, Vancouver, BC V5Z 1K3, Canada",49.263574,-123.125513,CA,4.5,7,BC
5,Pizzamia Gourmet Pizza Bar,"407 W Broadway, Vancouver, BC V5Y 1R4, Canada",49.263322,-123.113257,CA,4.5,7,BC
6,Blaze Gourmet Burgers,"43 E 5th Avenue, Vancouver, BC V5T 1G7, Canada",49.266644,-123.103743,CA,4.5,24,BC
7,Seaport City Seafood Restaurant,"2425 Cambie Street, Vancouver, BC V5Z 4M5, Canada",49.263921,-123.115152,CA,4.5,17,BC
8,Rolling Cashew,"401 Industrial Avenue, Vancouver, BC V6A 2P8, ...",49.269990,-123.095880,CA,4.5,19,BC
9,Superbaba,"2419 Main St, Vancouver, BC V5T 3E1, Canada",49.263330,-123.101268,CA,4.5,72,BC


### For better visualization and model building I decided to get some information via Yelp_API which is more related to the city_bike_df, above infomation is more related to the city.

#### Creating a df using latitude and longitude of the city_bike_df

In [116]:
len(city_bike_df) # length of city_bike_df
yelp_unique_places = [] 
for i in range(len(city_bike_df)):   
    latitude = city_bike_df['latitude'][i]
    longitude = city_bike_df['longitude'][i]
    
    url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius=1000&categories=&sort_by=best_match&limit=2"

    headers = {
    "Authorization": f"Bearer {yelp_api_key}",
    "Accept": "application/json"
    }

    result = requests.get(url, headers=headers)
         
   
    businesses = data['businesses']

    for business in businesses:
        name = business["name"]
        address = ", ".join(business["location"]["display_address"])
        latitude = business["coordinates"]["latitude"]
        longitude = business["coordinates"]["longitude"]
        country = business["location"]["country"]
        rating = business["rating"]
        review = business["review_count"]
        state = business["location"]["state"]
        distance = business["distance"]
        
        yelp_unique_places.append({
            "name": name,
            "latitude": latitude,
            "longitude": longitude,
            "distance" : distance,
            "yelp_rating": rating,
            "yelp_review": review,
        })
        
# Create DataFrame
df_yelp_unique = pd.DataFrame(yelp_unique_places)

# Print the DataFrame
df_yelp_unique

,name,latitude,longitude,distance,yelp_rating,yelp_review
0,Le Légendaire Restaurant Bar,48.410643,-71.053362,522.311253,4.0,6
1,Boefish,48.405576,-71.057046,787.422233,4.5,5
2,Passion Café,48.409290,-71.054947,552.408480,4.0,5
3,Rôtisserie St-Hubert,48.413318,-71.049191,799.101949,2.5,4
4,Ben & Florentine,48.403481,-71.054605,1066.122108,2.5,6
5,Le Légendaire Restaurant Bar,48.410643,-71.053362,522.311253,4.0,6
6,Boefish,48.405576,-71.057046,787.422233,4.5,5
7,Passion Café,48.409290,-71.054947,552.408480,4.0,5
8,Rôtisserie St-Hubert,48.413318,-71.049191,799.101949,2.5,4
9,Ben & Florentine,48.403481,-71.054605,1066.122108,2.5,6


Dataframe contains the places near to every single bike station. some of the values are duplicate as it is possible to have same place near differnet bike station. So, removing duplicates in the next step, befor that making sure that the given data is near the station.

In [117]:
df_yelp_unique['station_near_from'] = city_bike_df.apply(lambda row: f"{row['latitude']:.6f}, {row['longitude']:.6f}", axis=1)
df_yelp_unique.head()

,name,latitude,longitude,distance,yelp_rating,yelp_review,station_near_from
0,Le Légendaire Restaurant Bar,48.410643,-71.053362,522.311253,4.0,6,"48.428023, -71.062506"
1,Boefish,48.405576,-71.057046,787.422233,4.5,5,"48.404457, -71.058208"
2,Passion Café,48.409290,-71.054947,552.408480,4.0,5,"48.420480, -71.053380"
3,Rôtisserie St-Hubert,48.413318,-71.049191,799.101949,2.5,4,"48.337230, -70.879564"
4,Ben & Florentine,48.403481,-71.054605,1066.122108,2.5,6,"48.425119, -71.051336"


In [118]:
df_yelp_unique = df_yelp_unique.drop_duplicates()
df_yelp_unique.head()

,name,latitude,longitude,distance,yelp_rating,yelp_review,station_near_from
0,Le Légendaire Restaurant Bar,48.410643,-71.053362,522.311253,4.0,6,"48.428023, -71.062506"
1,Boefish,48.405576,-71.057046,787.422233,4.5,5,"48.404457, -71.058208"
2,Passion Café,48.409290,-71.054947,552.408480,4.0,5,"48.420480, -71.053380"
3,Rôtisserie St-Hubert,48.413318,-71.049191,799.101949,2.5,4,"48.337230, -70.879564"
4,Ben & Florentine,48.403481,-71.054605,1066.122108,2.5,6,"48.425119, -71.051336"


In [114]:
df_yelp_unique = pd.merge(city_bike_df, df_yelp_unique, on=['station_near_from'], how='left')
df_yelp_unique

,city,station_id,station,latitude_x,longitude_x,slots,empty_slots,free_bikes,station_near_from,name,latitude_y,longitude_y,distance,yelp_rating,yelp_review
0,Saguenay,b6903d788830595e054121e462598381,Place du Citoyen,48.428023,-71.062506,7,4,2,"48.428023, -71.062506",Le Légendaire Restaurant Bar,48.410643,-71.053362,522.311253,4.0,6
1,Saguenay,3aa299f4d3048ef94737bbf5fbc03825,Place du Royaume,48.404457,-71.058208,7,1,6,"48.404457, -71.058208",Boefish,48.405576,-71.057046,787.422233,4.5,5
2,Saguenay,0b1c0c48e94fae24b1a38f4559a5985a,UQAC,48.420480,-71.053380,7,5,2,"48.420480, -71.053380",Passion Café,48.409290,-71.054947,552.408480,4.0,5
3,Saguenay,f450af93c8d0f5ae5de8ea7cb1aafe70,Terminus La Baie,48.337230,-70.879564,7,5,2,"48.337230, -70.879564",Rôtisserie St-Hubert,48.413318,-71.049191,799.101949,2.5,4
4,Saguenay,7ada5d893c40c9a46a54f771a392db6f,Cégep de Chicoutimi,48.425119,-71.051336,7,5,2,"48.425119, -71.051336",Ben & Florentine,48.403481,-71.054605,1066.122108,2.5,6
5,Saguenay,989fc50e15bad6b1f40d26f986d50ccc,Station Rene-Levesque,48.404255,-71.216372,7,7,0,"48.404255, -71.216372",Le Légendaire Restaurant Bar,48.410643,-71.053362,522.311253,4.0,6
6,Saguenay,e5b2a5bf7b36bcd5d52ca0d63a56ae06,Terminus Jonquiere,48.411807,-71.254890,7,3,4,"48.411807, -71.254890",Boefish,48.405576,-71.057046,787.422233,4.5,5


latitude_x and longitude_x is courdinates of bike station, latitude_y and longitude_y is cordinate of POIs.

In [119]:
# Save the DataFrame as a CSV file in the data folder
import os

folder_path = r"C:\Users\akash\Desktop\BootCamp\Project_2\Statistical-Modelling-with-Python-main\data"
file_name = "poi_info.csv"
file_path = os.path.join(folder_path, file_name)
df_yelp_unique.to_csv(file_path, index=False)